<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.856 · Análisis de sentimientos y redes sociales</p>
<p style="margin: 0; text-align:right;">Máster universitario de Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# Análisis de sentimientos y redes sociales
## PLA4: Representación de redes sociales como grafos

## Uso de grafos para representar datos de redes sociales

En el capítulo 2 del libro *Análisis de datos de redes sociales*, *Introducción a la teoría de grafos*, hemos descrito qué son los grafos, los diferentes tipos de grafos que existen y las propiedades básicas que tienen. En este notebook, veremos cómo podemos representar en forma de grafo los datos de redes sociales que hemos adquirido con anterioridad (usando cualquiera de los tres métodos descritos en el notebook anterior) mediantre la librería de Python [Networkx](https://networkx.github.io/).

## 1. Representación básica 

[Networkx](https://networkx.github.io/) es una la librería de Python para crear y estudiar redes. Dispone de clases propias para representar los grafos, que ya incluyen métodos que implantan algunos de los algoritmos más comunes para analizar, visualizar y exportar grafos.

Networkx dispone de funciones de importación que permiten crear un grafo a partir de los datos de un fichero de texto con unos formatos concretos. Así, por ejemplo, los datos de Google+ que hemos descargado anteriormente del [Network Repository](http://networkrepository.com/soc.php) se encontraban en formato "lista de aristas", donde cada línea del fichero describe una arista del grafo. Podemos usar la función [`read_edgelist`](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/generated/networkx.readwrite.edgelist.read_edgelist.html#networkx.readwrite.edgelist.read_edgelist) para crear un grafo de Networkx a partir del fichero descargado:

In [1]:
# Importamos la librería networkx
import networkx as nx

# Abrimos el fichero descargado
with open("./data/soc-gplus.edges", 'rb') as gp_file:
    
    # Creamos el grafo
    g = nx.read_edgelist(gp_file, comments="%")
    
    # Mostramos el número de nodos y aristas del grafo
    print("El grafo tiene {} nodos y {} aristas".format(
        g.number_of_nodes(), g.number_of_edges()))


El grafo tiene 23628 nodos y 39194 aristas


Fijaos que es necesario especificar el carácter que se usa para indicar qué líneas del fichero corresponden a comentarios para lograr que el grafo se cargue correctamente ya que, por defecto, la función espera almohadillas `#` y el fichero que hemos importado usa `%`.

Además de leer una lista de aristas con `read_edgelist`, Networkx dispone de otras funciones de carga de datos de ficheros que permiten importar ficheros en otros formatos. Algunos de los más populares son [GML](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/generated/networkx.readwrite.gml.read_gml.html#networkx.readwrite.gml.read_gml), [graphml](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/generated/networkx.readwrite.graphml.read_graphml.html#networkx.readwrite.graphml.read_graphml), o [gexf](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/generated/networkx.readwrite.gexf.read_gexf.html#networkx.readwrite.gexf.read_gexf). En la documentación de Networkx encontraréis la [lista completa de formatos](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/index.html) para los cuáles existen funciones de importación.

Si no disponemos de un fichero con todos los datos del grafo, podemos optar por crear un grafo vacío e ir añadiendo sus nodos y aristas a medida que vamos obteniendo los datos. Esta sería una opción para crear un grafo a partir de datos que nos vayan devolviendo las sucesivas consultas a una API.

Las clases [`Graph`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/graph.html#networkx.Graph) y [`DiGraph`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/digraph.html#networkx.DiGraph) representan grafos simples no-dirigidos y dirigidos, respectivamente. Los métodos [`add_node`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/generated/networkx.Graph.add_node.html#networkx.Graph.add_node) y [`add_edge`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/generated/networkx.Graph.add_edge.html#networkx.Graph.add_edge) permiten añadir un nodo o una arista a ambos tipos de grafos. 

Veamos un ejemplo de cómo podemos crear un grafo con datos de relaciones de seguimiento de usuarios de Twitter usando las funciones de la API de Twitter que hemos visto en el notebook anterior y la clase `DiGraph` de Networkx que acabamos de presentar.

En primer lugar, recuperamos el código de autenticación de la API de Twitter:

In [2]:
# Importamos la librería Tweepy
import tweepy

# Definimos la variable api, que almacenará el objeto tweepy.API inicializado con nuestras credenciales
api = None

# Definimos la función get_twitter_api, que devolverá el objeto tweepy.API. La función creará
# un objeto tweepy.API si este no se ha creado previamente (o si lo indicamos explícitamente 
# con reset=True) o bien devolverá el objeto creado anteriormente.
def get_twitter_api(reset=False, consumer_key=None, consumer_secret=None, access_token=None, access_secret=None):
    global api
    if not api or reset:
        print("Initializing tweepy API...")
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        api = tweepy.API(auth)
    return api

# Creamos un objeto tweepy.API con la función get_twitter_api
# IMPORTANTE: es necesario incluir las credenciales de acceso que hayáis obtenido al crear vuestra App
# para ejecutar el resto de ejemplos del notebook.
api = get_twitter_api(consumer_key = 'tuzlLXJ7Cjcw9ysoeEhT60ENh', 
                      consumer_secret = 'cO9hqRaP94IP66ssVqHHqVyibrLMQ1et8XHbhczy2eBW6rFFiI',
                      access_token = '1029309220234186752-GikHFK2syTi2kiH0bVx8fMNhdP8IDJ',
                      access_secret = 'AJIuznR3PXwPh30GZ8YYqyQfpTyjkZrnu5bglzGUSaN5F'
                      )


Initializing tweepy API...


Después, implementamos dos funciones que incorporan la lógica de nuestro script. La primera, `add_connections_from`, recupera los *followers* y los *friends* de un usuario de Twitter y añade los nodos y las aristas al grafo. La segunda, `bfs_from_seed`, implementa una búsqueda *Breadth First Search* a partir de un nodo inicial, que recibe como parámetro (consultad el capítulo 2 del libro *Análisis de datos de redes sociales* si no recordáis cómo funciona este algoritmo).

In [3]:
def add_connections_from(g, new_nodes, userid):
    """
    Adds followers and friends from user with id userid to the graph g.
    
    :param g: networkx graph
    :param new_nodes: list of nodes where newly discovered nodes will be added
    :param userid: integer, Twitter user id of the node that will be visited
    """
    
    try:
        # get userid's user followers
        followers = api.followers_ids(userid)
        print("{} followers retrieved".format(len(followers)))
        for follower in followers:
            # if the follower is unknown, add follower to the graph and to new_nodes list
            if follower not in g.nodes():
                g.add_node(follower)
                new_nodes.append(follower)
            # add edge to the graph
            g.add_edge(userid, follower)

        # get userid's user friends
        friends = api.friends_ids(userid)
        print("{} friends retrieved".format(len(friends)))
        for friend in friends:
            # if the friend is unknown, add friend to the graph and to new_nodes list
            if friend not in g.nodes():
                g.add_node(friend)
                new_nodes.append(friend)
            # add edge to the graph
            g.add_edge(friend, userid)
    except tweepy.TweepError:
        # If we can not obtain data from this user, skip it
        print("Failed to run the command on that user, skipping...")
    except tweepy.RateLimitError:
        # If we hit the rate limit, wait for 15 minutes before calling the API again
        print("Rate limit reached. Waiting for 15 minutes...")
        time.sleep(15 * 60)

    return g, new_nodes

def bfs_from_seed(seed_id, number_of_crawled_users=3):
    """
    Performs a bfs search starting from user seed_id and stopping after having
    processed number_of_crawled_users users.
    
    :param seed_id: int, user id of the starting user
    :param number_of_crawled_users: int, number of users to crawl
    """

    print("Starting to crawl at node {}\n".format(seed_id))
    
    # create an empty graph and add seed node
    g = nx.DiGraph()
    g.add_node(seed_id)
    
    # create a list to store newly discovered users
    new_nodes = []
    
    # crawl number_of_crawled_users users
    for _ in range(number_of_crawled_users):
        print("New user processed {}".format(seed_id))
        # add friends of followers of crawled user to the graph
        g, new_nodes = add_connections_from(g, new_nodes, seed_id)
        print("Waiting list has now {} users".format(len(new_nodes)))
        print("Graph has now {} nodes and {} edges\n".format(
            g.number_of_nodes(), g.number_of_edges()))
        # get next user to crawl
        seed_id = new_nodes.pop(0)
        
    return g, new_nodes


In [4]:
bfs_from_seed(2775195514)

Starting to crawl at node 2775195514

New user processed 2775195514
1760 followers retrieved
144 friends retrieved
Waiting list has now 1824 users
Graph has now 1825 nodes and 1904 edges

New user processed 1037262684029370368
55 followers retrieved
278 friends retrieved
Waiting list has now 2098 users
Graph has now 2100 nodes and 2236 edges

New user processed 232668523
Failed to run the command on that user, skipping...
Waiting list has now 2097 users
Graph has now 2100 nodes and 2236 edges



(<networkx.classes.digraph.DiGraph at 0x117479d68>,
 [1119737786557456385,
  799963701604454400,
  795676354977079296,
  15039203,
  140694740,
  1144221712336244736,
  2325217039,
  219635897,
  97070739,
  2410354324,
  2396037342,
  2312793855,
  1140319788306259969,
  1067809214624669697,
  426698976,
  813611378,
  203652936,
  1339878091,
  1000249440765169665,
  103377307,
  132204002,
  4155536837,
  1164611,
  103131326,
  862918790958518273,
  4897204209,
  364449057,
  761783419,
  207994318,
  292609406,
  1005786690353598465,
  289291773,
  4053496954,
  302207857,
  223082075,
  636126399,
  632844131,
  245022854,
  1109816501455020033,
  2778890025,
  1001155508,
  4811665277,
  109899149,
  1131566718118244352,
  706450767021002752,
  409395187,
  196665734,
  4369594635,
  860127057522688000,
  969251488781434880,
  2242599391,
  1130832192706416640,
  299526368,
  129596166,
  315376679,
  104653002,
  2331890599,
  260355276,
  1127935596863873024,
  106556090677879

## 2. Grafos con atributos

Tanto los grafos como las aristas y los nodos pueden contener atributos en Networkx. Estos atributos pueden usarse, por ejemplo, para almacenar metadatos del grafo, pesos o *timestamps* en las aristas, propiedades de los nodos, etc. 

Los atributos pueden asignarse en el momento de la creación del grafo, el nodo o la arista, o bien con posterioridad:

In [5]:
# Creamos el grafo g con los atributos 'network' y 'seed'
g = nx.Graph(network='Twitter', seed=2775195514)
print('Los atributos del grafo son: {}'.format(g.graph))
print('El atributo network tiene como valor: {}'.format(g.graph['network']))
print('El atributo seed tiene como valor: {}'.format(g.graph['seed']))

Los atributos del grafo son: {'network': 'Twitter', 'seed': 2775195514}
El atributo network tiene como valor: Twitter
El atributo seed tiene como valor: 2775195514


In [6]:
# Creamos un grafo g vacío y añadimos después los atributos 'network' y 'seed'
g = nx.Graph()
print('Los atributos del grafo son: {}'.format(g.graph))
g.graph['network'] = "Twitter"
g.graph['seed'] = 2775195514
print('Los atributos del grafo son: {}'.format(g.graph))

Los atributos del grafo son: {}
Los atributos del grafo son: {'network': 'Twitter', 'seed': 2775195514}


In [7]:
# Añadimos un nodo con atributos al grafo
g.add_node(1, day='monday')
print('Los atributos de los nodos son: {}'.format(g.nodes(data=True)))

Los atributos de los nodos son: [(1, {'day': 'monday'})]


In [8]:
# Añadimos un nodo sin atributos y los asignamos después
g.add_node(2)
print('Los atributos de los nodos son: {}'.format(g.nodes(data=True)))
g.nodes[2]['day'] = 'tuesday'
print('Los atributos de los nodos son: {}'.format(g.nodes(data=True)))

Los atributos de los nodos son: [(1, {'day': 'monday'}), (2, {})]
Los atributos de los nodos son: [(1, {'day': 'monday'}), (2, {'day': 'tuesday'})]


In [9]:
# Añadimos una arista con atributos al grafo
g.add_edge(1, 2, friendship_created_at='2018-07-15')
print('Los atributos de las aristas son: {}'.format(g.edges(data=True)))

Los atributos de las aristas son: [(1, 2, {'friendship_created_at': '2018-07-15'})]


In [10]:
# Añadimos una arista sin atributos y los asignamos después
g.add_node(3)
g.add_edge(1, 3)
print('Los atributos de las aristas son: {}'.format(g.edges(data=True)))
g[1][3]['friendship_created_at'] = '2018-04-23'
print('Los atributos de las aristas son: {}'.format(g.edges(data=True)))

Los atributos de las aristas son: [(1, 2, {'friendship_created_at': '2018-07-15'}), (1, 3, {})]
Los atributos de las aristas son: [(1, 2, {'friendship_created_at': '2018-07-15'}), (1, 3, {'friendship_created_at': '2018-04-23'})]


Fijaos que para poder añadir una segunda arista al grafo, hemos tenido que añadir un tercer nodo. Esto es así porque la clase [`Graph`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/graph.html#networkx.Graph) representa grafos simples, en los cuales no se admiten aristas repetidas. Para representar las relaciones de amistad de una red social, normalmente nos será suficiente usar este tipo de grafos. De todos modos, en ocasiones puede ser necesario permitir aristas repetidas. Este es el caso, por ejemplo, de un grafo que represente *replies* entre usuarios de Twitter: un usuario puede responder más de una vez a otro usuario. La clase [`MultiGraph`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/multigraph.html#multigraph-undirected-graphs-with-self-loops-and-parallel-edges) de networkx nos permite representar estos grafos.

In [11]:
# Creamos un multigrafo
g = nx.MultiGraph()

# añadimos dos nodos y dos aristas entre ellos
g.add_nodes_from([1,2])
g.add_edge(1,2, reply_sent_at='2018-07-15')
g.add_edge(1,2, reply_sent_at='2018-07-18')
print('Las aristas son: {}'.format(g.edges(data=True)))

Las aristas son: [(1, 2, {'reply_sent_at': '2018-07-15'}), (1, 2, {'reply_sent_at': '2018-07-18'})]


Aunque Networkx no tiene soporte nativo para grafos dinámicos, la clase `MultiGraph` puede ser útil para representar estos grafos. Usando atributos, podemos especificar los momentos en los que los nodos y las aristas son creados y/o eliminados del grafo. 